# Dataset Genration for Q1

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

transactions_data = [
    ("t1", "u1", "p1", "$120.50", "USD", "2025-02-01 10:15:00"),
    ("t2", "u2", "p2", "$200.00", "USD", "01-02-2025 11:20:00"),  # inconsistent date
    ("t3", None, "p3", "$50.00", "USD", "2025/02/01 12:00:00"),  # null user
    ("t4", "u3", "p4", "-$30.00", "USD", "2025-02-01 13:00:00"), # negative
    ("t5", "u1", "p5", "$75.00", "USD", "2025-02-01 14:00:00"),
    ("t5", "u1", "p5", "$75.00", "USD", "2025-02-01 14:05:00"),  # duplicate (later timestamp)
    ("t6", "u4", "p2", "$500.00", "USD", "2025-02-02 09:00:00"),
    ("t7", "u2", "p3", "$300.00", "USD", "2025-02-02 10:30:00"),
    ("t8", "u3", "p1", "$150.00", "USD", "2025-02-02 11:00:00"),
]

transactions_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("amount", StringType(), True),
    StructField("currency", StringType(), True),
    StructField("transaction_timestamp", StringType(), True),
])

transactions_df = spark.createDataFrame(transactions_data, schema=transactions_schema)

transactions_df.write.mode("overwrite").option("header", True).csv("/Volumes/pyspark/bronze/raw_ingestion/csv/transactions_csv")


In [0]:
users_data = [
    ("u1", "uk", "2024-01-01"),
    ("u2", "Us", "2024-02-01"),
    ("u3", "IN", "2024-03-01"),
    ("u3", "in", "2024-03-02"),  # duplicate user (latest signup)
    ("u4", None, "2024-04-01"),  # null country
]

users_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("country", StringType(), True),
    StructField("signup_date", StringType(), True),
])

users_df = spark.createDataFrame(users_data, schema=users_schema)

users_df.write.mode("overwrite").option("header", True).csv("/Volumes/pyspark/bronze/raw_ingestion/csv/users_csv")


# Pair Programming Task — End-to-End PySpark Pipeline
---
## Scenario

You are given two datasets:

### 📁 `transactions.csv`

```
transaction_id (string)
user_id (string)
product_id (string)
amount (string)
currency (string)
transaction_timestamp (string)
```

Problems:

* amount contains currency symbols (e.g. "$120.50")
* timestamp format inconsistent
* duplicates exist
* some null user_id rows
* some negative amounts

---

### 📁 `users.csv`

```
user_id (string)
country (string)
signup_date (string)
```

Problems:

* duplicate users
* country casing inconsistent
* null values

---




## Step 1 — Read Data

* Read both datasets
* Define schema explicitly
* Handle malformed rows safely

---

In [0]:
schema_trans = "transaction_id string, user_id string, product_id string, amount string, currency string, transaction_timestamp string"

schema_user  = "user_id string, country string, signup_date string"

read_opts = {
    "header": "true",
    "mode": "PERMISSIVE",
    "columnNameOfCorruptRecord": "_corrupt_record"
}

df_trans = (
    spark.read
      .options(**read_opts)
      .schema(schema_trans)
      .csv("/Volumes/pyspark/bronze/raw_ingestion/csv/transactions_csv/")
)

df_user = (
    spark.read
      .options(**read_opts)
      .schema(schema_user)
      .csv("/Volumes/pyspark/bronze/raw_ingestion/csv/users_csv/")
)

In [0]:
from pyspark.sql.types import _parse_datatype_string as p

schema_trans = p(schema_trans)

In [0]:
df_trans.limit(5).display()
df_user.limit(5).display()

## Step 2 — Clean Data

### Transactions:

* Remove null user_id
* Remove negative amounts
* Clean amount → cast to double
* Standardise timestamp → proper timestamp type
* Deduplicate based on latest transaction_timestamp per transaction_id



In [0]:
from pyspark.sql.functions import regexp_replace, col, coalesce, expr

df_clean = (df_trans
  .withColumn("amount", regexp_replace(col("amount"), r"\$", "").cast("double"))
  .withColumn(
      "transaction_timestamp",
      coalesce(
          expr("try_to_timestamp(transaction_timestamp, 'dd-MM-yyyy HH:mm:ss')"),
          expr("try_to_timestamp(transaction_timestamp, 'yyyy-MM-dd HH:mm:ss')")
      )
  )
  .filter(col("amount") > 0)
  .filter(col("user_id").isNotNull())
)

In [0]:
from pyspark.sql.functions import regexp_replace, col, coalesce, expr, row_number
from pyspark.sql.window import Window

wind_spec = Window.partitionBy("transaction_id").orderBy(col("transaction_timestamp").desc())

tran_final = df_clean.withColumn("rank", row_number().over(wind_spec)).filter(col("rank")==1).drop("rank")
tran_final.display()

### Users:

* Deduplicate based on latest signup_date
* Standardise country to uppercase

---

In [0]:
from pyspark.sql.functions import upper

wind_spec_user = Window.partitionBy("user_id").orderBy(col("signup_date").desc())

user_final = df_user.withColumn("rank",row_number().over(wind_spec_user)).filter("rank==1").drop("rank")\
    .withColumn("country", upper(col("country")))

## Step 3 — Transform

1. Join transactions with users
2. Calculate:

   * Daily revenue per country
3. Return:

   * transaction_date
   * country
   * total_revenue

---


In [0]:
from pyspark.sql.functions import to_date, sum

df = tran_final.join(user_final, ["user_id"], "inner")
df = df.withColumn("date", to_date(col("transaction_timestamp")))

df.groupBy("country","date")\
    .agg(sum("amount").alias("total_revenue"))\
    .orderBy(col("total_revenue").desc())\
    .display()

## Step 4 — Advanced Requirement (Very Likely)

For each country and day:

* Return top 3 users by revenue

Output:

```
transaction_date
country
user_id
total_user_revenue
rank
```

---

In [0]:
user_total_rev = df.groupBy("user_id","date","country")\
    .agg(sum(col("amount")).alias("total_user_revenue"))

user_total_rev.display()

In [0]:
window_spec = Window.partitionBy("country","date").orderBy(col("total_user_revenue").desc())
user_total_rev = (
    user_total_rev.withColumn(
        "rank", row_number().over(window_spec)
    )
    .filter(col("rank")<= 3)
)


In [0]:
user_total_rev.display()


## Step 5 — Performance Discussion

After coding they will ask:

* What causes shuffle here?
* Where would you broadcast?
* How would you optimise for 1 billion rows?
* How would you partition when writing?
* How would you productionise this?

---